## synthetic Hartmann6 test function. 
we will attempt to maximize $-f(x)$ to achieve $\max_{x} -f(x) = 3.32237$.

In [1]:
import os
import inspect
import torch
from botorch.test_functions import Hartmann
from botorch.models import SingleTaskGP
from botorch.acquisition.analytic import ExpectedImprovement,ProbabilityOfImprovement,UpperConfidenceBound
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_mll,fit_fully_bayesian_model_nuts # TODO
from botorch.optim import optimize_acqf
import warnings
warnings.filterwarnings('ignore')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double

In [2]:
# BO parameter - tunable
acq_fun = ProbabilityOfImprovement
acq_kwargs = {} # conditional para UpperConfidenceBound - 'beta':0.5
fit = fit_gpytorch_mll

# fixed
q = 1
init_n = 10
num_restarts = 4
raw_samples = 128
Bo_iter = 50
verbose = 10 # int number of iter to show best f so far

In [3]:
def initialize_model(x, y, fit, state_dict=None):
    model = SingleTaskGP(x, y).to(device)
    mll = ExactMarginalLogLikelihood(model.likelihood, model).to(device)
    if state_dict is not None:
        model.load_state_dict(state_dict)
    fit(mll);
    return model

In [4]:
# problem parameter
noise = 0.1
neg_hartmann6 = Hartmann(negate=True)
neg_hartmann6_noise = Hartmann(noise_std=noise, negate=True)
bounds = torch.tensor([[0.0] * 6, [1.0] * 6], device=device, dtype=dtype)

In [5]:
# init x,y
x = torch.rand(init_n,6,device=device,dtype=dtype)
y = neg_hartmann6_noise(x)[:,None]

In [6]:
model = initialize_model(x, y, fit)
for j in range(1,1+Bo_iter):
    # set up acqucision fun
    if 'best_f' in inspect.signature(acq_fun).parameters:
        acq_kwargs['best_f'] = y.max().item()
    acq = acq_fun(model,**acq_kwargs)

    # optimize over x_next
    x_next = optimize_acqf(acq,bounds,q=q,num_restarts=num_restarts,raw_samples=raw_samples)[0].detach()

    # try x_next
    y_next = neg_hartmann6_noise(x_next)

    # update dataset
    x = torch.cat([x,x_next])
    y = torch.cat([y,y_next[:,None]])

    # update model
    model = initialize_model(x, y, fit, model.state_dict())
    
    if j%verbose == 0:
        # select from existing x, better for less noisy problem
#         argmax_i = y.argmax().item()
#         y_best = neg_hartmann6(x[argmax_i])
        # select from mean of posterior
        mean_fun = lambda x:model(x).mean
        x_best = optimize_acqf(mean_fun,bounds,q=1,num_restarts=1,raw_samples=1)[0].detach()
        y_best =  neg_hartmann6(x_best)
        print('best val is {} at iter {}'.format(y_best.item(),j))

best val is tensor([0.2578], device='cuda:0', dtype=torch.float64) at iter 10
best val is tensor([0.0216], device='cuda:0', dtype=torch.float64) at iter 20
best val is tensor([3.0044], device='cuda:0', dtype=torch.float64) at iter 30
best val is tensor([2.9784], device='cuda:0', dtype=torch.float64) at iter 40
best val is tensor([3.0451], device='cuda:0', dtype=torch.float64) at iter 50
